In [1]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain_qdrant import QdrantVectorStore


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from qdrant_client import QdrantClient,models

import os
import openai
from tqdm import tqdm
import pickle
import re
from datasets import Dataset
from ragas import evaluate

/Users/domenico/Desktop/RAG_BGG/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
openai_emb = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"],model="text-embedding-ada-002")
openai_model =  ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"],model="gpt-4o-mini", temperature=0.15)

/var/folders/f1/rbp6fp3946nb5h41g3zj6x0m0000gn/T/ipykernel_6470/3177344771.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  openai_emb = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"],model="text-embedding-ada-002")
/var/folders/f1/rbp6fp3946nb5h41g3zj6x0m0000gn/T/ipykernel_6470/3177344771.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  openai_model =  ChatOpe

In [16]:
TITLE = 'Skull King'

# Dataset Creation
Creo un dataset di query chiedendo un LLM di generare domande sul gioco scelto partendo dai chunk

In [17]:
filter = models.Filter(
        must=[
            models.FieldCondition(
                key='metadata.title',
                match=models.MatchValue(value=TITLE),
            )
        ]
    )

In [18]:
#creo una lista di chunk con relativi metadati
client = QdrantClient(url="192.168.1.7:6333")
points = client.scroll(collection_name="bg",scroll_filter=filter)

chunks = [] 
while points:
    for point in points[0]:
        payload = point.payload
        chunks.append({"page_content":payload["page_content"],"metadata":payload["metadata"]})
    if points[1]:
        points = client.scroll(collection_name="bg",offset=points[1],scroll_filter=filter)
    else:
        break

In [19]:
len(chunks)

40

In [20]:
#setting LLM
template = """
You are an assistant specialized in analyzing texts for board games.
You will be provided with an excerpt of text (chunk) related to the rules, descriptions, or other elements of a board game, along with its contextual metadata.
Your task is to generate a set of questions based exclusively on the rules of the game described in the chunk and metadata, ensuring the questions are strictly related to the game and not to any other unrelated topics that might appear in the text.

Instructions:
 - Generate a variable number of questions depending on the complexity and amount of information in the chunk.
 - he questions must concern only the rules, mechanics, or elements of the game described in the input. Ignore any other unrelated topics that may appear in the text.
 - If the chunk is too short or does not contain sufficient information about the game’s rules, return "No questions can be generated."
 - Do not add, invent, or infer anything that is not explicitly stated in the chunk and metadata.
 - Formulate clear, concise, and relevant questions that help clarify or explore the game’s rules and mechanics.
Provided Input:
 - Text Chunk: {page_content}
 - Metadata: {metadata}
Expected Output:
 - A list of relevant questions based solely on the game’s rules and mechanics as described in the provided chunk and metadata.
 - If no questions can be generated, return: "No questions can be generated."

"""

prompt = ChatPromptTemplate.from_template(template)



chain = (
    {"page_content":RunnablePassthrough(), "metadata": RunnablePassthrough()}
    | prompt
    | openai_model
    | StrOutputParser()
)


In [21]:
file_name_queries = "_".join(el.lower() for el in TITLE.split(" "))+"_queries.pkl"
file_name_answer = "_".join(el.lower() for el in TITLE.split(" "))+"_answer.pkl"

In [22]:
queries = []
for chunk in tqdm(chunks):
    res = chunk
    resp = chain.invoke({"page_content":chunk["page_content"],"metadata":str(chunk["metadata"])})
    res["queries"] = resp
    queries.append(res)

  0%|          | 0/40 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [62]:
# with open("skull_king_queries.pkl","wb") as f:
#     pickle.dump(queries,f)

# Answers generation

In [7]:
with open("skull_king_queries.pkl","rb") as f:
    queries = pickle.load(f)

In [8]:
queries

[{'page_content': "# The Rascal'S Scoring  \n![17_Image_0.Png](17_Image_0.Png)",
  'metadata': {'Header 1': "The Rascal'S Scoring", 'title': 'Skull King'},
  'queries': 'No questions can be generated.'},
 {'page_content': "## Suit Cards  \nThere are 4 suits of cards numbered 1-14 in the deck. There are three standard suits; Parrot (green), Treasure  \n![7_image_0.png](7_image_0.png) Chest (yellow), Treasure Map (purple), and the trump suit: Jolly Roger (Black). Jolly Roger cards outrank (trump) the other three suits.  \n8 If a suit card is played first in a trick (lead), all players must 'follow suit' and play that same suit (if they are going to play a numbered card). If you don't have the suit that was lead, you may play any other suit. If all cards played are the same suit, the highest numbered card would win the trick. Cards without numbers are not suit cards and may be played regardless of what was led.  \nExample: Samuel plays a green 7, Bonny then lays down a green 12, and Henry

In [11]:
qdrant = QdrantVectorStore.from_existing_collection(collection_name="bg",embedding=openai_emb,url="192.168.1.7:6333")
retriever = qdrant.as_retriever(search_kwargs={"k":5,"fetch_k":10,"lambda_mult":0.9},search_type="mmr")

In [36]:
def format_docs(docs):
    return "\n\n".join(doc for doc in docs)


template = """
You are an expert assistant specializing in board games. Your role is to provide authoritative, precise, and practical guidance on game rules, mechanics, strategies, and scenarios. 
You respond as the ultimate reference for the games discussed, ensuring clarity and correctness. Your answers should feel as though they’re guiding the player through a live game session. 
Avoid general advice or unrelated topics. Instead, focus entirely on providing rule explanations, strategic insights, and in-game examples based on the player's current scenario.

The game you're explaining today is: **{title}**

---
**Current Situation**:  
This is the specific context or scenario the player is in, which might affect your answer:  
_{context}_

---
**Player's Question**:  
_{question}_

---
**Response**:  
Provide your answer in an instructive and conversational tone as if you’re explaining the rules and strategies at the table. Include relevant examples, clarify mechanics, and offer advice on how to best handle the current scenario:

- **Game Rule Explanation**: Offer precise details on the relevant game rules, mechanics, or actions related to the question.
  
- **Contextual Strategy/Advice**: If applicable, give strategic advice based on the player’s current in-game context, During this contextualization, do not give example of a specific case, just be vague for some strategy applicable in general, not in the specific case, unless explicity asked so.

- **Example**: Where useful, provide an example to illustrate the explanation more clearly.
"""

prompt = ChatPromptTemplate.from_template(template)



chain = (
    {"title":RunnablePassthrough(), "context": RunnablePassthrough() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | openai_model
    | StrOutputParser()
)

In [42]:
answers = []
for query in tqdm(queries):
    questions_string = query["queries"]
    questions = questions_string.split("\n")
    for question in questions:
        if question == "No questions can be generated.":
            continue
        question = re.sub(r"([0-9]\. )","",question)
        contexts = [doc.page_content for doc in retriever.invoke(question)]
        answer = chain.invoke({"title":"Skull King", "context": contexts, "question": question})
        answers.append({"contexts": contexts, "question": question,"answer":answer})

100%|██████████| 40/40 [29:58<00:00, 44.96s/it]


In [43]:
# with open("skull_king_answers.pkl","wb") as f:
#     pickle.dump(answers,f)

In [46]:
eval_dataset = Dataset.from_list(answers)

In [62]:
from ragas.metrics import LLMContextPrecisionWithoutReference,ContextEntityRecall,Faithfulness,AnswerRelevancy,AnswerSimilarity,AnswerCorrectness

In [69]:
metrics = [LLMContextPrecisionWithoutReference(),Faithfulness(),AnswerRelevancy()]

In [70]:
results = evaluate(dataset=eval_dataset,metrics=metrics,llm=openai_model,embeddings=openai_emb)

Evaluating: 100%|██████████| 492/492 [12:35<00:00,  1.54s/it]


In [71]:
df = results.to_pandas()

In [73]:
df.to_csv("results.csv")

In [74]:
df

,user_input,retrieved_contexts,response,llm_context_precision_without_reference,faithfulness,answer_relevancy
0,"How many suits of cards are there in the game,...",[## Suit Cards \nThere are 4 suits of cards n...,"In **Skull King**, there are **four suits of c...",1.0,1.000000,0.881135
1,What is the ranking of the Jolly Roger suit co...,[## Suit Cards \nThere are 4 suits of cards n...,"In **Skull King**, the Jolly Roger suit (black...",1.0,0.916667,0.894069
2,What must players do if a suit card is played ...,[## Leading With Special Cards \nWhen you lea...,When a suit card is played first in a trick in...,1.0,0.411765,0.846428
3,What happens if a player does not have the sui...,[#### Leading With An Escape \nWhen an Escape...,"In **Skull King**, if a player does not have t...",1.0,1.000000,0.859082
4,How is the winner of a trick determined when a...,[# Key Terms \nFor those unfamiliar with thes...,"In **Skull King**, when all cards played in a ...",1.0,0.826087,0.949986
...,...,...,...,...,...,...
159,Can you provide an example of how scoring work...,[#### Bidding Zero \nBid zero and get your bi...,"In **Skull King**, when a player bids zero, th...",1.0,1.000000,0.863677
160,What happens when the Kraken and the White Wha...,[#### Roiling Waters \nThe Kraken and the Whi...,When the Kraken and the White Whale are played...,1.0,0.892857,0.973364
161,How does the second card played in a trick aff...,[#### Roiling Waters \nThe Kraken and the Whi...,"In **Skull King**, when the Kraken and the Whi...",1.0,0.676471,0.908106
162,What action is set by the card that wins the b...,[#### Roiling Waters \nThe Kraken and the Whi...,"In **Skull King**, when the Kraken and the Whi...",1.0,0.521739,0.883682


In [75]:
df.describe()

,llm_context_precision_without_reference,faithfulness,answer_relevancy
count,164.000000,164.000000,164.000000
mean,0.991717,0.739538,0.908573
std,0.042155,0.219784,0.041547
min,0.679167,0.150000,0.802224
25%,1.000000,0.577068,0.877547
50%,1.000000,0.796552,0.909740
75%,1.000000,0.922932,0.936663
max,1.000000,1.000000,1.000000


In [4]:
from evaluation import generate_eval_dataset
from vector_db import QdrantVectorDB

In [ ]:
URL = ""
API_KEY = ""

In [5]:
db = QdrantVectorDB(URL,API_KEY)

In [6]:
generate_eval_dataset(model=openai_model,emb=openai_emb,db=db,collection_name="bg",title="Skull King")

2024-12-30 11:38:50,650 - INFO - Generating queries....
100%|██████████| 5/5 [00:13<00:00,  2.64s/it]
2024-12-30 11:39:03,851 - INFO - Queries generated.
2024-12-30 11:40:32,206 - INFO - Generating answers...
 20%|██        | 1/5 [01:04<04:18, 64.67s/it]


ResponseHandlingException: Server disconnected without sending a response.